In [ ]:
!pip install pathway bokeh panel --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import pathway as pw
import datetime
import bokeh.plotting
import panel as pn
from google.colab import files


In [ ]:
uploaded = files.upload()


Saving dataset.csv to dataset.csv


In [ ]:
df = pd.read_csv("dataset.csv")

# ✅ If your raw data uses TrafficConditionNearby, map it
if "TrafficConditionNearby" in df.columns:
    df["Traffic"] = df["TrafficConditionNearby"]

# ✅ Map Traffic to numeric if needed
traffic_map = {"low": 1.0, "average": 2.0, "high": 3.0}
df["Traffic"] = df["Traffic"].map(traffic_map).fillna(1.0)

# ✅ UNIX timestamp
df["Timestamp"] = pd.to_datetime(
    df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
    format="%d-%m-%Y %H:%M:%S"
)
df["TimestampSeconds"] = df["Timestamp"].astype("int64") // 10**9

# ✅ Fill any missing QueueLength
if "QueueLength" not in df.columns:
    df["QueueLength"] = 5

if "IsSpecialDay" not in df.columns:
    df["IsSpecialDay"] = 0

if "VehicleType" not in df.columns:
    df["VehicleType"] = "car"

# ✅ Force final CSV save with EXACT names
df[[
    "TimestampSeconds",
    "Occupancy",
    "Capacity",
    "QueueLength",
    "Traffic",
    "IsSpecialDay",
    "VehicleType"
]].to_csv("parking_stream_model2.csv", index=False)

# ✅ Check it!
!head parking_stream_model2.csv


TimestampSeconds,Occupancy,Capacity,QueueLength,Traffic,IsSpecialDay,VehicleType
1475567940,61,577,1,1.0,0,car
1475569500,64,577,1,1.0,0,car
1475571540,80,577,2,1.0,0,car
1475573520,107,577,2,1.0,0,car
1475575140,150,577,2,1.0,0,bike
1475576760,177,577,3,1.0,0,car
1475578740,219,577,6,3.0,0,truck
1475580300,247,577,5,2.0,0,car
1475582340,259,577,5,2.0,0,cycle


In [ ]:
class ParkingSchema2(pw.Schema):
    TimestampSeconds: int
    Occupancy: int
    Capacity: int
    QueueLength: int
    Traffic: float
    IsSpecialDay: int
    VehicleType: str


In [ ]:
data = pw.demo.replay_csv(
    "parking_stream_model2.csv",
    schema=ParkingSchema2,
    input_rate=1000
)


In [ ]:
data_with_time = data.with_columns(
    timestamp_parsed = data.TimestampSeconds,
    occ = data.Occupancy,
    cap = data.Capacity,
    queue = data.QueueLength,
    traffic = data.Traffic,
    special = data.IsSpecialDay,
    vtype = data.VehicleType
)


In [ ]:
@pw.udf
def vehicle_weight(vtype: str) -> float:
    weights = {"car": 1.0, "bike": 0.5, "truck": 1.5}
    return weights.get(vtype, 1.0)

ALPHA, BETA, GAMMA, DELTA, EPSILON = 1.0, 0.5, 0.3, 0.2, 0.4

demand = data_with_time.with_columns(
    occ_rate = pw.this.occ / pw.this.cap,
    vweight = vehicle_weight(pw.this.vtype),
    demand_raw = (
        ALPHA * (pw.this.occ / pw.this.cap)
        + BETA * pw.this.queue
        - GAMMA * pw.this.traffic
        + DELTA * pw.this.special
        + EPSILON * vehicle_weight(pw.this.vtype)
    )
)


In [ ]:
micro = (
    demand.windowby(
        pw.this.timestamp_parsed,
        window=pw.temporal.tumbling(1800)
    )
    .reduce(
        demand_min = pw.reducers.min(pw.this.demand_raw),
        demand_max = pw.reducers.max(pw.this.demand_raw),
        t = pw.reducers.max(pw.this.timestamp_parsed),
        demand_sum = pw.reducers.sum(pw.this.demand_raw),
        demand_count = pw.reducers.count(1)
    )
)

micro = micro.with_columns(
    demand_norm = pw.apply(
        lambda avg, dmin, dmax: 0.0 if (dmax - dmin) == 0 else (avg - dmin) / (dmax - dmin),
        pw.this.demand_sum / pw.this.demand_count,
        pw.this.demand_min,
        pw.this.demand_max
    )
)



In [ ]:

BASE_PRICE = 10.0
LAMBDA = 1.0

micro = micro.with_columns(
    price = pw.apply(
        lambda norm: BASE_PRICE * (1 + LAMBDA * norm),
        pw.this.demand_norm
    )
)



In [ ]:
pn.extension()

def model2_plotter(source):
    fig = bokeh.plotting.figure(
        height=400, width=800, title="Model 2: Demand-Based Price", x_axis_type="linear"
    )
    fig.line("t", "price_bounded", source=source, line_width=2, color="blue")
    fig.scatter("t", "price_bounded", source=source, size=6, color="red")
    return fig

viz = micro.plot(model2_plotter, sorting_col="t")


In [ ]:
pw.io.csv.write(micro, "model2_output.csv")


/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [ ]:
%%capture --no-display
pw.run()


Output()